Lab setup - follow these steps to successfully complete the lab:
1. Please download each of the 4 csv files in this github repo: https://github.com/dobbinjp/squirrel_data/tree/main
2. Enable DBFS browsing in Advanced Settings (click on your user icon, select "settings" then "advanced" and search for "DBFS")
3. Go to your catalog and click on DBFS then "Upload"
4. Specify a folder ("squirrel" or something similar) and browse to upload the 4 files
5. Copy the folder location to your clipboard and paste it in the text box at the top of this notebook ("folder_loc")
6. Specify a value for "dbname" and run this notebook ("Run All" button should work)

In [0]:
# copy the location of the folder with your squirrel data (from DBFS)
# set the database_name and squirrel_folder variables to the current parameter text
squirrel_folder = dbutils.widgets.get("folder_loc")
database_name = dbutils.widgets.get("dbname")

In [0]:
# store list of files in a variable
squirrel_datafiles = dbutils.fs.ls(squirrel_folder)

In [0]:
# print the paths of each file in the DBFS squirrel data folder
for s in squirrel_datafiles:
  # dbutils.fs.rm(s.path) # this line can be uncommented if you need to clear the DBFS folder
  print(s)

FileInfo(path='dbfs:/FileStore/squirrel123/park_data.csv', name='park_data.csv', size=4204, modificationTime=1744145433000)
FileInfo(path='dbfs:/FileStore/squirrel123/squirrel_data_2018.csv', name='squirrel_data_2018.csv', size=720561, modificationTime=1744145433000)
FileInfo(path='dbfs:/FileStore/squirrel123/squirrel_data_2020.csv', name='squirrel_data_2020.csv', size=59517, modificationTime=1744145432000)
FileInfo(path='dbfs:/FileStore/squirrel123/stories.csv', name='stories.csv', size=7995, modificationTime=1744145432000)


In [0]:
# create the database if it does not exist already, then set it to the current database
spark.sql(f"create database if not exists {database_name};")
spark.sql(f"use {database_name};")

DataFrame[]

In [0]:
# loop through all data files, read them and make a table of each
for i in squirrel_datafiles:
  print(i.name)
  df = spark.read.format("csv").options(inferSchema=True, header=True).load(i.path)
  spark.sql(f"drop table if exists {i.name.replace('.csv','')};")
  df.write.format("delta").mode("overwrite").saveAsTable(i.name.replace('.csv',''))

park_data.csv
squirrel_data_2018.csv
squirrel_data_2020.csv
stories.csv


In [0]:
%sql
--enable change data feed on the table we will be manipulating
ALTER TABLE squirrel_data_2020 SET TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
%sql
  update squirrel_data_2020
  set Longitude = -1 * Longitude
  where Longitude > 0;

num_affected_rows
1


In [0]:
%sql

delete from squirrel_data_2020 
where Longitude is null and Latitude is null;

num_affected_rows
92


In [0]:
%sql

insert into
  squirrel_data_2020
select
  null as AreaName,
  null as AreaID,
  null as ParkName,
  null as ParkID,
  UniqueSquirrelID as SquirrelID,
  PrimaryFurColor,
  HighlightFurColor as HighlightsinFurColor,
  Colornotes as ColorNotes,
  Location,
  AboveGroundSighterMeasurement as HeightAboveGroundFeet,
  null as SpecificLocation,
  OtherActivities as Activities,
  OtherInteractions as InteractionswithHumans,
  null as Notes,
  Y as Latitude,
  X as Longitude
from
  squirrel_data_2018;

num_affected_rows,num_inserted_rows
3023,3023


In [0]:
%sql
select * from squirrel_data_2020;

AreaName,AreaID,ParkName,ParkID,SquirrelID,PrimaryFurColor,HighlightsinFurColor,ColorNotes,Location,HeightAboveGroundFeet,SpecificLocation,Activities,InteractionswithHumans,Notes,Latitude,Longitude
null,null,null,null,37F-PM-1014-03,null,null,null,null,null,null,null,null,null,40.79408239,-73.95613449
null,null,null,null,21B-AM-1019-04,null,null,null,null,null,null,null,null,null,40.78378252,-73.96885747
null,null,null,null,11B-PM-1014-08,Gray,null,null,Above Ground,10,null,null,null,null,40.77553362,-73.97428115
null,null,null,null,32E-PM-1017-14,Gray,null,Nothing selected as Primary. Gray selected as Highlights. Made executive adjustments.,null,null,null,null,null,null,40.79031289,-73.95964139
null,null,null,null,13E-AM-1017-05,Gray,Cinnamon,null,Above Ground,null,null,null,null,null,40.77621269,-73.97026765
null,null,null,null,11H-AM-1010-03,Cinnamon,White,null,null,null,null,null,null,null,40.77259088,-73.96836135
null,null,null,null,36H-AM-1010-02,Gray,null,just outside hectare,Ground Plane,FALSE,null,null,null,null,40.79318117,-73.95412018
null,null,null,null,33F-AM-1008-02,Gray,null,null,Ground Plane,FALSE,null,null,null,null,40.79173678,-73.95826943
null,null,null,null,21C-PM-1006-01,Gray,null,null,Ground Plane,FALSE,null,null,null,null,40.78297239,-73.9674286
null,null,null,null,11D-AM-1010-03,Gray,Cinnamon,null,Above Ground,30,null,grooming,null,null,40.77428796,-73.97225002


In [0]:
%sql
describe history squirrel_data_2020;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2025-04-08T21:55:13Z,1392220080137065,jimmy.dobbins@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3703707432810307),0408-204609-c3omxhhv,3,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3023, numOutputBytes -> 89822)",null,Databricks-Runtime/15.4.x-scala2.12
3,2025-04-08T21:55:10Z,1392220080137065,jimmy.dobbins@gmail.com,DELETE,"Map(predicate -> [""(isnull(Longitude#21494) AND isnull(Latitude#21493))""])",null,List(3703707432810307),0408-204609-c3omxhhv,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 20776, numCopiedRows -> 341, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 1, executionTimeMs -> 1645, numDeletionVectorsUpdated -> 0, numDeletedRows -> 92, scanTimeMs -> 679, numAddedFiles -> 1, numAddedBytes -> 19718, rewriteTimeMs -> 966)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-04-08T21:55:05Z,1392220080137065,jimmy.dobbins@gmail.com,UPDATE,"Map(predicate -> [""(Longitude#20478 > 0.0)""])",null,List(3703707432810307),0408-204609-c3omxhhv,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 20549, numCopiedRows -> 432, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 1, executionTimeMs -> 2900, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1382, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 20776, rewriteTimeMs -> 1517)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-04-08T21:55:00Z,1392220080137065,jimmy.dobbins@gmail.com,SET TBLPROPERTIES,"Map(properties -> {""delta.enableChangeDataFeed"":""true""})",null,List(3703707432810307),0408-204609-c3omxhhv,0,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12
0,2025-04-08T21:54:54Z,1392220080137065,jimmy.dobbins@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(3703707432810307),0408-204609-c3omxhhv,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 433, numOutputBytes -> 20549)",null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
select count(*) from squirrel_data_2020 version as of 4;

count(1)
3364


In [0]:
%sql

select * from table_changes('squirrel_data_2020', 3)

AreaName,AreaID,ParkName,ParkID,SquirrelID,PrimaryFurColor,HighlightsinFurColor,ColorNotes,Location,HeightAboveGroundFeet,SpecificLocation,Activities,InteractionswithHumans,Notes,Latitude,Longitude,_change_type,_commit_version,_commit_timestamp
UPPER MANHATTAN,A,Highbridge Park,3,A-03-13,Gray,Cinnamon,null,Ground Plane,null,null,Running,null,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-01,Gray,Gray,null,Ground Plane,null,null,"Running, up tree",Runs From,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-02,Gray,Gray,null,Ground Plane,null,null,"Running, up tree",Runs From,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-03,Gray,Gray,null,Ground Plane,null,null,"Running, up tree",Indifferent,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-04,Gray,Gray,null,Ground Plane,null,null,Chasing,Indifferent,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-05,Gray,Gray,null,Ground Plane,null,null,Chasing,null,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-06,Gray,Gray,null,Above Ground,15,null,Sitting on branch,Indifferent,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-07,Gray,Gray,null,Ground Plane,null,null,"Running, up tree",Indifferent,Forever wild section,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-08,Gray,Gray,null,Ground Plane,null,null,Eating,Indifferent,null,null,null,delete,3,2025-04-08T21:55:10Z
UPPER MANHATTAN,A,Riverside Park (Section Near Grant Memorial),5,A-05-09,Gray,Gray,null,Ground Plane,null,null,Eating,Indifferent,null,null,null,delete,3,2025-04-08T21:55:10Z
